In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time


NbClients = 5
clientLocs = []

for i in range(NbClients+1):
    clientLocs.append([random.randint(0, NbClients*10), random.randint(0, NbClients*10)])

clientLocs = np.array(clientLocs)

print("Client Locations:\n", clientLocs)

# calcule la matrice des distances entre les clients
def distance_matrix(locs):
    n = locs.shape[0] # Calcul du nombre de clients
    dist_matrix = np.zeros((n, n)) # Création d'une matrice n x n remplie de zéros
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = np.linalg.norm(locs[i] - locs[j]) # Calcul de la distance euclidienne entre les clients i et j
        dist_matrix = np.round(dist_matrix).astype(int) # Arrondi des distances et conversion en entiers

    return dist_matrix

matrix = distance_matrix(clientLocs)
print("Distance Matrix:\n", distance_matrix(clientLocs))


Client Locations:
 [[39 28]
 [ 8  8]
 [ 0  0]
 [13 49]
 [13 10]
 [10 18]]
Distance Matrix:
 [[ 0 37 48 33 32 31]
 [37  0 11 41  5 10]
 [48 11  0 51 16 21]
 [33 41 51  0 39 31]
 [32  5 16 39  0  9]
 [31 10 21 31  9  0]]


In [ ]:
filename = f"matrice_distances_{NbClients+1}x{NbClients+1}.csv"  
np.savetxt(filename, matrix, delimiter=",", fmt='%d')


In [ ]:
listeClient = [5, 8, 3, 6, 12]
depot = 0


def voisinMinPoid(matrice, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrice[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrice[cur][i]
        elif matrice[cur][i] > 0 and matrice[cur][i] < poidMinTrajet:
            poidMinTrajet = matrice[cur][i]
            nextVoisin = i

    return nextVoisin

def voisinsClientGrapheMatrice(matrice, sommet):
    voisins = [i for i in listeClient if matrice[sommet][i] > 0]
    return voisins 

def poidCycle(matrice, cycle):
    poids_total = 0
    for i in range(len(cycle) - 1):
        poids_total += matrice[cycle[i]][cycle[i + 1]]
    poids_total += matrice[cycle[-1]][cycle[0]] 
    return poids_total



def recherche_tabou_cycle(matrice, start, firstNeighbor, iter_max=100):

    # On copie la matrice pour ne pas modifier l’originale
    matrice_copy = copy.deepcopy(matrice)

    # Le cycle que nous construisons (liste d’indices de sommets)
    cycle = [start]

    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(listeClient) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)

    # Le sommet courant (celui où on se trouve actuellement)
    cur = start

    # Boucle principale de la recherche tabou
    for _ in range(iter_max):

        if len(cycle) == 1:
            voisin = firstNeighbor
        else:
            voisins = voisinsClientGrapheMatrice(matrice_copy, cur) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin = voisinMinPoid(matrice_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrice_copy[cur][voisin] = 0
        matrice_copy[voisin][cur] = 0
        
        cycle.append(voisin) # On ajoute ce voisin au cycle    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur = voisin

    # On retourne le chemin (cycle) trouvé
    return cycle



def tabou_multi_start(matrice, nb_lancements=10, iter_max=100):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    - iter_max : nombre d’itérations par recherche
    """

    meilleur_cycle = []  # Le meilleur cycle global (le plus court)
    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(nb_lancements):

        start = depot

        firstNeighbor = 0

        while matrice[start][firstNeighbor] == 0:
            firstNeighbor = random.randint(0, len(listeClient)-1)

        # On effectue une recherche tabou locale à partir de ce sommet
        cycle = recherche_tabou_cycle(matrice, start, firstNeighbor, iter_max)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1}: départ={firstNeighbor}, longueur du cycle={len(cycle)}, temps du trajet={poidCycle(matrice, cycle)}")

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle(matrice, cycle)
            meilleur_cycle = cycle
            goodI = i+1
        elif poidCycle(matrice, cycle) < tempsMeilleurCycle:
            meilleur_cycle = cycle
            tempsMeilleurCycle = poidCycle(matrice, cycle)
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return meilleur_cycle, tempsMeilleurCycle, goodI



matrice = [
 [0, 12, 5, 19, 1, 8, 16, 11, 6, 7, 13, 17, 10],
 [12, 0, 4, 3, 15, 18, 20, 8, 19, 11, 14, 5, 9],
 [5, 4, 0, 14, 17, 10, 2, 13, 7, 18, 8, 9, 16],
 [19, 3, 14, 0, 12, 6, 9, 20, 15, 2, 10, 11, 7],
 [1, 15, 17, 12, 0, 13, 11, 3, 8, 4, 16, 14, 5],
 [8, 18, 10, 6, 13, 0, 5, 9, 20, 16, 3, 15, 19],
 [16, 20, 2, 9, 11, 5, 0, 7, 14, 19, 18, 12, 4],
 [11, 8, 13, 20, 3, 9, 7, 0, 10, 17, 2, 5, 15],
 [6, 19, 7, 15, 8, 20, 14, 10, 0, 12, 9, 16, 18],
 [7, 11, 18, 2, 4, 16, 19, 17, 12, 0, 15, 8, 6],
 [13, 14, 8, 10, 16, 3, 18, 2, 9, 15, 0, 19, 20],
 [17, 5, 9, 11, 14, 15, 12, 5, 16, 8, 19, 0, 13],
 [10, 9, 16, 7, 5, 19, 4, 15, 18, 6, 20, 13, 0]
]


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")

# Lancement du multi-start (10 essais, taille tabou = 5, 100 itérations max)
meilleur_cycle, tempsMeilleurCycle, goodI = tabou_multi_start(matrice, nb_lancements=10, iter_max=100)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "Longueur du cycle :", len(meilleur_cycle), "  Temps du cycle :", tempsMeilleurCycle)
for s in meilleur_cycle:
    print(s + 1, "-> ", end='')
print(meilleur_cycle[0]+1)  # on revient au départ pour fermer le cycle

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")

prob_bouchon = 0.3
facteur = 3



def generer_facteur_bouchon(heure):
    """
    Génère un facteur global de bouchon selon l'heure de la journée.
    - Peu de bouchons la nuit
    - Maximal vers 8h et 17h
    """
    # Heure normalisée sur 24h → sinus pour faire un cycle
    intensite = 0.5 + 0.5 * math.sin((heure - 8) / 24 * 2 * math.pi)
    # Variation entre 1.0 et 3.0 environ
    n = random.uniform(-2,2)
    facteur = n + 2.0 * intensite  
    if facteur <= 0:
        facteur = 1
    return facteur

def cout_effectif(i, j, heure):
    """
    Retourne le coût dynamique entre 2 villes à une heure donnée.
    Complexité O(1)
    """
    base = matrice[i][j]
    if base == 0:
        return 0
    
    # Facteur global du trafic (selon l'heure)
    facteur_temps = generer_facteur_bouchon(heure)
    
    # Petite variation pseudo-aléatoire stable (chaque route a sa propre sensibilité)
    # Cela évite que tout augmente/diminue uniformément
    random.seed(i * 1000 + j)  # stable entre runs
    variation = random.uniform(-0.1, 0.1)  # entre -10% et +10%
    
    cout = base * facteur_temps * (1 + variation)
    return round(cout, 2)

def simulation_journee():
    """
    Simule 24 heures avec bouchons évoluant toutes les 4 heures.
    """
    heures = list(range(0, 25, 4))  # 0h, 4h, 8h, 12h, 16h, 20h, 24h
    for h in heures:
        print(f"\n=== Heure {h}h ===")
        facteur = generer_facteur_bouchon(h)
        print(f"Facteur global de bouchon : {facteur:.2f}")
        
        # Exemple : coût de la route (0 -> 5) évolue dans la journée
        cout_05 = cout_effectif(0, 5, h)
        print(f"Coût entre ville 0 et 5 à {h}h : {cout_05}")


simulation_journee()



In [ ]:
import numpy as np

